In [6]:
import numpy as np
import pandas as pd

def load_sess_trials(cache, sess_id, stim_label_list, bin_size, kernel_param):
    # load session data
    data = cache.get_session_data(sess_id)
    for i, stim_label in enumerate(stim_label_list):
        # get start, end time points
        stim_data_table = data.get_stimulus_table()[data.get_stimulus_table().stimulus_name==stim_label]
        start_time = np.array(stim_data_table.start_time)
        stop_time = np.array(stim_data_table.stop_time)
        trial_time = np.mean(stop_time - start_time)
        bin_edges = np.arange(trial_time + bin_size, step = bin_size )
        t_bin[i] = bin_edges
        stim_presentation_ids = data.stimulus_presentations.loc[
            (data.stimulus_presentations['stimulus_name'] == stim_label)
            ].index.values
        unit_ids = data.units.index.values
        P[i] = presentationwise_spike_counts(bin_edges,stim_presentation_ids,unit_ids)
        # get windowed spike times
        # S[i] = window_spk_times(data,start_time,stop_time)
        # get binned spike rates
        # t_bin[i], P[i] = bin_spikes(S[i],bin_size)
        # kernel smoothed estimate of binned spike rates
        X[i] = smooth_spikes(P[i],kernel_param)
        
    return (t_bin, P, X)

In [7]:
def window_spk_times( data, start_time, stop_time ):
#     # Vincent
#     ## The code I suppose
#     unit_spikes=data.spike_times[data.units.index.values]
#     S = unit_spikes[np.logical_and(unit_spikes>=start_time,unit_spikes<stop_time)]
#     ## alternative code
#     drifting_gratings_presentation_ids = session.stimulus_presentations.loc[(session.stimulus_presentations['stimulus_name'] == 'drifting_gratings')].index.values
#     S = session.presentationwise_spike_times(stimulus_presentation_ids=drifting_gratings_presentation_ids,
#                                          unit_ids=data.units.index.value,
#                                          np.logical_and(unit_spikes>=start_time,unit_spikes<stop_time))
    
    return S 

In [8]:
def bin_spikes( S, bin_size, trial_time ):
    # Ryan
    P = list()
    t_bin = np.arange(trial_time, step = bin_size)
    for s_trial in S:
        pi = list()
        for unit in s_trial:
            pj, _ = np.histogram(np.array(unit), bins = t_bin)
            pi.append(pj)
        P.append(pi)
    return (t_bin, P)

In [9]:
def smooth_spikes( P, kernel_param ):
    # Rahul
    """
    P: #trials x #units x #time-points sized ndarray
    kernel_param: width of kernel (in samplingrate*1000 units)

    """

    import numpy as np
    import time
    import scipy.stats, scipy.signal
    import scipy as sp

    start = time.time() # tic Measures function running speed. Works only on Macs

    trials = P.shape[0]
    units = P.shape[1]
    bins = P.shape[2]

    # Create fake data for testing purposes
    #    array = np.random.randint(0,2,(trials,bins))
    #    sigma = 45

    #Define kernel
    sigma = kernel_param/6. 
    edges = np.arange(-3*sigma,3*sigma+.001,.001)
    kernel = sp.stats.norm.pdf(edges,0, sigma) #Use a gaussian function

    #Compute Spike Density Function for all trials
    X = np.zeros((trials,units,bins))
    X = sp.signal.convolve(P, kernel[None,None, :], mode='same')

    print('Run time for SDF function was ' + str(round(time.time()-start)) + 'seconds') # toc
    return X